In [6]:
from fmpy import *
from fmpy.fmi3 import *

filename = 'BouncingBall.fmu'


f = open('BouncingBall_out.csv', 'w')

f.write('time,h,v\n')

model_description = read_model_description(filename)

vr = dict((v.name, v.valueReference) for v in model_description.modelVariables)

unzipdir = extract(filename)

fmu_instance = instantiate_fmu()

def intermediate_update(instanceEnvironment: fmi3InstanceEnvironment,
                       intermediateUpdateTime: fmi3Float64,
                       intermediateVariableSetRequested: fmi3Boolean,
                       intermediateVariableGetAllowed: fmi3Boolean,
                       intermediateStepFinished: fmi3Boolean,
                       canReturnEarly: fmi3Boolean,
                       earlyReturnRequested: POINTER(fmi3Boolean),
                       earlyReturnTime: POINTER(fmi3Float64)) -> None:
    
    
    
    print(intermediateUpdateTime)
    
    

    earlyReturnRequested.contents = fmi3False


result = simulate_fmu(filename, output_interval=0.1, intermediate_update=intermediate_update)

plot_result(result, markers=True)

0.001
0.002
0.003
0.004
0.005
0.006
0.007
0.008
0.009000000000000001
0.01
0.011
0.012
0.013000000000000001
0.014
0.015
0.016
0.017
0.018000000000000002
0.019
0.02
0.021
0.022
0.023
0.024
0.025
0.026000000000000002
0.027
0.028
0.029
0.03
0.031
0.032
0.033
0.034
0.035
0.036000000000000004
0.037
0.038
0.039
0.04
0.041
0.042
0.043000000000000003
0.044
0.045
0.046
0.047
0.048
0.049
0.05
0.051000000000000004
0.052000000000000005
0.053
0.054
0.055
0.056
0.057
0.058
0.059000000000000004
0.06
0.061
0.062
0.063
0.064
0.065
0.066
0.067
0.068
0.069
0.07
0.07100000000000001
0.07200000000000001
0.073
0.074
0.075
0.076
0.077
0.078
0.079
0.08
0.081
0.082
0.083
0.084
0.085
0.08600000000000001
0.08700000000000001
0.088
0.089
0.09
0.091
0.092
0.093
0.094
0.095
0.096
0.097
0.098
0.099
0.1
0.101
0.10200000000000001
0.10300000000000001
0.10400000000000001
0.105
0.106
0.107
0.108
0.109
0.11
0.111
0.112
0.113
0.114
0.115
0.116
0.117
0.11800000000000001
0.11900000000000001
0.12
0.121
0.122
0.123
0.124
0.125
0.

1.74
1.741
1.742
1.743
1.744
1.745
1.746
1.747
1.748
1.749
1.75
1.7510000000000001
1.752
1.7530000000000001
1.754
1.7550000000000001
1.756
1.7570000000000001
1.758
1.7590000000000001
1.76
1.7610000000000001
1.762
1.7630000000000001
1.764
1.7650000000000001
1.766
1.7670000000000001
1.768
1.7690000000000001
1.77
1.7710000000000001
1.772
1.7730000000000001
1.774
1.7750000000000001
1.776
1.7770000000000001
1.778
1.7790000000000001
1.78
1.7810000000000001
1.782
1.7830000000000001
1.784
1.7850000000000001
1.786
1.7870000000000001
1.788
1.7890000000000001
1.79
1.7910000000000001
1.792
1.793
1.794
1.795
1.796
1.797
1.798
1.799
1.8
1.801
1.802
1.803
1.804
1.805
1.806
1.807
1.808
1.809
1.81
1.811
1.812
1.813
1.814
1.815
1.816
1.817
1.818
1.819
1.82
1.821
1.822
1.823
1.824
1.825
1.826
1.827
1.828
1.829
1.83
1.831
1.832
1.833
1.834
1.835
1.836
1.837
1.838
1.839
1.84
1.841
1.842
1.843
1.844
1.845
1.846
1.847
1.848
1.849
1.85
1.851
1.852
1.853
1.854
1.855
1.856
1.857
1.858
1.859
1.86
1.861
1.862
1.8